In [ ]:
pip install feedparser
import requests
from bs4 import BeautifulSoup
import feedparser
import csv
import pandas as pd
import re

# источники
our_feeds = {'Klerk': 'https://www.klerk.ru/export/news.rss',
'Glavbukh': 'https://www.glavbukh.ru/rss/news.xml',
'Buh':'https://buh.ru/rss/?chanel=articles',
'Garant': 'http://rss.garant.ru/categories/news/',
'Nalogi' : 'http://rss.garant.ru/consult/nalog/',
'MK' : 'https://www.mk.ru/rss/economics/index.xml'} 

#пример пути файла
f_all_news = 'allnews23march.csv' 
f_certain_news = 'certainnews23march.csv'

#пример таргетов
vector1 = 'ДолЛАР|РубЛ|ЕвРО'
vector2 = 'ЦБ|СбЕРбАНК|курс'

def check_url(url_feed): #функция получает линк на рсс ленту, возвращает        
# распаршенную ленту с помощью feedpaeser
    return feedparser.parse(url_feed)  
    
def getHeadlines(url_feed): #функция для получения заголовков новости
    headlines = []
    lenta = check_url (url_feed)
    for item_of_news in lenta['items']:
        headlines.append(item_of_news ['title'])
    return headlines

def getDescriptions(url_feed): #функция для получения описания новости
    descriptions = []
    lenta = check_url(url_feed)
    for item_of_news in lenta['items']:
        descriptions.append(item_of_news ['description'])
    return descriptions

def getLinks(url_feed): #функция для получения ссылки на источник новости
    links = []
    lenta = check_url(url_feed)
    for item_of_news in lenta['items']:
        links.append(item_of_news ['link'])
    return links

def getDates(url_feed): #функция для получения даты публикации новости
    dates = []
    lenta = check_url(url_feed)
    for item_of_news in lenta['items']:
        dates.append(item_of_news ['published'])
    return dates

allheadlines = []
alldescriptions = []
alllinks = []
alldates = []

# Прогоняем URL и добавляем их в пустые списки
for key,url in our_feeds.items():
    allheadlines.extend( getHeadlines(url) )
    
for key,url in our_feeds.items():
    alldescriptions.extend( getDescriptions(url) )
    
for key,url in our_feeds.items():
    alllinks.extend( getLinks(url) )
    
for key,url in our_feeds.items():
    alldates.extend( getDates(url) )

def write_all_news(all_news_filepath): #функция для записи всех новостей в .csv, 
# возвращает нам этот датасет
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
        
            
    return df

def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #функция для поиска, а затем записи
                #определенных новостей по таргета,
                #затем возвращает этот датасет
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

df = write_all_news(f_all_news) #все новости
df.to_csv(r'/content/export_dataframe.csv', index = False, header=True, encoding='utf-8')

looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) 